# Create one common dataset for footprint, EC, biomet and continuous soil T data

This notebook was created because there were missing data on T<sub>air</sub> between 13/9-16/9 2022

02/2024

In [1]:
import pandas as pd
from dotenv import load_dotenv; load_dotenv()
from datetime import datetime, timedelta
import os

import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import xarray as xr

from utils import load_Ran_ec_data, load_Ran_biomet_data

sns.set_context('notebook', font_scale=1.5)

%matplotlib widget
%load_ext autoreload
%autoreload 2

## Load EC and biomet flux array

In [2]:
SOILCLASSIFICATION_DATA = os.getenv('SOILCLASSIFICATION_DATA')
ONEDRIVE_FOLDER = os.getenv('ONEDRIVE_FOLDER')

In [3]:
biomet = load_Ran_biomet_data(ONEDRIVE_FOLDER, [2021, 2022])

/Users/opa/Ranskalankorpi_CC_EC/utils.py:49: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  dataframes.append(pd.read_csv(path(years[0]), skiprows=[1], index_col=0, date_parser=lambda x: pd.to_datetime(x)))
/Users/opa/Ranskalankorpi_CC_EC/utils.py:52: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  dataframes.append(pd.read_csv(path(year), skiprows=[1], index_col=0, date_parser=lambda x: pd.to_datetime(x)))


In [4]:
ec = load_Ran_ec_data(ONEDRIVE_FOLDER, [2021, 2022])
tt = ec.time.values
# remove dim_0 and set previous time values to time coordinates
ec = ec.drop_dims('dim_0').assign_coords(time=tt)

### Set all non-zero N<sub>2</sub>O fluxes to nan and all CH<sub>4</sub> fluxes smaller than -10 nmol/m2/s to nan

added 7.6.2024

In [ ]:
ec.ch4_flux_final.where()

In [15]:
gg = ec.ch4_flux_final.copy()

In [19]:
gg < -10/1000

<xarray.DataArray 'ch4_flux_final' (time: 18835)>
array([False, False, False, ..., False, False, False])
Coordinates:
  * time     (time) datetime64[ns] 2021-11-24T13:30:00 ... 2022-12-31T23:30:0...

In [33]:
ec.ch4_flux_final[ec.ch4_flux_final < -10/1000] = np.nan

In [34]:
ec.n2o_flux_final[ec.n2o_flux_final < 0] = np.nan

## Create one consistent time vector

In [39]:
time_all = pd.date_range(datetime(2021,11,24,0,0,0), datetime(2022,12,31, 23, 30, 0), freq='30min')

In [40]:
time_all

DatetimeIndex(['2021-11-24 00:00:00', '2021-11-24 00:30:00',
               '2021-11-24 01:00:00', '2021-11-24 01:30:00',
               '2021-11-24 02:00:00', '2021-11-24 02:30:00',
               '2021-11-24 03:00:00', '2021-11-24 03:30:00',
               '2021-11-24 04:00:00', '2021-11-24 04:30:00',
               ...
               '2022-12-31 19:00:00', '2022-12-31 19:30:00',
               '2022-12-31 20:00:00', '2022-12-31 20:30:00',
               '2022-12-31 21:00:00', '2022-12-31 21:30:00',
               '2022-12-31 22:00:00', '2022-12-31 22:30:00',
               '2022-12-31 23:00:00', '2022-12-31 23:30:00'],
              dtype='datetime64[ns]', length=19344, freq='30T')

## Get common indeces with EC

In [41]:
ec_time = ec.time.to_dataframe()

In [42]:
ind_ec_in_all = time_all.get_indexer(ec_time.index, method='nearest')

In [43]:
df_all = pd.DataFrame(index=time_all)

In [44]:
df_all

""
2021-11-24 00:00:00
2021-11-24 00:30:00
2021-11-24 01:00:00
2021-11-24 01:30:00
2021-11-24 02:00:00
...
2022-12-31 21:30:00
2022-12-31 22:00:00
2022-12-31 22:30:00
2022-12-31 23:00:00


In [45]:
xr_all = xr.Dataset(coords=dict(
    time=time_all
))

In [46]:
for key in ec.keys():
    temp_var = np.zeros((df_all.shape[0],))*np.nan
    temp_var[ind_ec_in_all] = ec[key].values
    xr_all[key] = ("time", temp_var)

## Set all non-zero N<sub>2</sub>O fluxes to nan and all CH<sub>4</sub> fluxes smaller than -10 nmol/m2/s to nan

added 7.6.2024

## Get common indeces with biomet

In [47]:
biomet_time = biomet.time.to_dataframe()

In [48]:
ind_biomet_in_all = time_all.get_indexer(biomet.time, method='nearest')

In [49]:
for key in biomet.keys():
    temp_var = np.zeros((df_all.shape[0],))*np.nan
    temp_var[ind_biomet_in_all] = biomet[key].values
    xr_all[key] = ("time", temp_var)

## Load fpr contribution array

In [50]:
fpr_cont = xr.open_dataset('data/footprint/footprint_soilclass_contribution_2022.nc')

In [51]:
fpr_cont

<xarray.Dataset>
Dimensions:           (soil_class: 10, time: 17055)
Coordinates:
  * soil_class        (soil_class) <U16 'Fallen tree' ... 'Instruments'
  * time              (time) datetime64[ns] 2022-01-01 ... 2022-12-31T23:30:00
Data variables:
    fpr_contribution  (soil_class, time) float64 ...
Attributes:
    description:  soil class contribution to footprint

### assign new coordinate to xr_all "soil_class"

In [52]:
xr_all = xr_all.assign_coords(soil_class=fpr_cont.soil_class)

In [53]:
ind_fpr_in_all = time_all.get_indexer(fpr_cont.time, method='nearest')

In [54]:
for key in fpr_cont.keys():
    temp_var = np.zeros((xr_all.soil_class.shape[0], df_all.shape[0]))*np.nan
    temp_var[:, ind_fpr_in_all] = fpr_cont[key].values
    xr_all[key] = (["soil_class", "time"], temp_var)

In [55]:
xr_all

<xarray.Dataset>
Dimensions:            (time: 19344, soil_class: 10)
Coordinates:
  * time               (time) datetime64[ns] 2021-11-24 ... 2022-12-31T23:30:00
  * soil_class         (soil_class) <U16 'Fallen tree' ... 'Instruments'
Data variables: (12/83)
    H                  (time) float64 nan nan nan nan ... -32.69 -30.69 -31.54
    H_qf               (time) float64 nan nan nan nan ... -32.69 -30.69 -31.54
    H_final            (time) float64 nan nan nan nan ... -32.69 -30.69 -31.54
    rand_err_H         (time) float64 nan nan nan nan ... 4.997 6.068 6.291
    LE                 (time) float64 nan nan nan nan ... -22.21 -12.28 -14.01
    LE_qf              (time) float64 nan nan nan nan ... nan -12.28 -14.01
    ...                 ...
    TS_1_1_1           (time) float64 nan nan nan nan nan ... 1.6 1.6 1.6 1.6
    TS_2_1_1           (time) float64 nan nan nan nan ... 2.677 2.673 2.687
    TS_3_1_1           (time) float64 nan nan nan nan ... 3.413 3.403 3.4 3.4
    WTD_1_1_1          (time) float64 nan nan nan nan ... 1.55 1.55 1.55 1.55
    PA_1_1_1           (time) float64 nan nan nan nan ... 97.18 97.22 97.23
    fpr_contribution   (soil_class, time) float64 nan nan ... 4.689e-09

## WTD

In [56]:
def date_parser_wtd(row):
    # There are four types of string that can come from row. This need to handled separately
    # The idea is to first parse only the date from both options (first 10 characters) and then extract the time of day separately
    # For each case that was found
    # First one is 2020-06-02 00:00:00 20:00:00
    # The second one is e.g., 2020-07-05 00:00:00 1900-02-02 18:00:00
    # The third is 2020-07-05 00:00:00 1900-02-02 18:00:00.59.590000
    # the fouth is 2020-07-05 00:00:00 1900-02-02 24:00:00

    # Take date
    date = datetime.strptime(row[:10], r'%Y-%m-%d')

    # check if there is . in the time

    if "." in row:
        t = row[20:28]
    else:
        t = row[-8:]
    
    t_split = t.split(":")
    hours = int(t_split[0])
    minutes = int(t_split[1])

    if hours == 0 and minutes == 0:
        hours = 24
        minutes = 0

    date = date + timedelta(hours=hours, minutes=minutes)

    return date

In [57]:
wtd = pd.read_excel(f'{ONEDRIVE_FOLDER}/WTD/ranskalankorpi_odyssey_data/Odyssey_wt-data_ranskalankorpi_2020-05-12_2023-04-27.xlsx', skiprows=4, header=0, parse_dates={'datetime': ['Date','Time']}, date_parser= lambda x: date_parser_wtd(x), index_col='datetime')

/var/folders/_7/7fgf24sd5ggbyr5x9kzrwz7c0000gn/T/ipykernel_18766/1193253790.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  wtd = pd.read_excel(f'{ONEDRIVE_FOLDER}/WTD/ranskalankorpi_odyssey_data/Odyssey_wt-data_ranskalankorpi_2020-05-12_2023-04-27.xlsx', skiprows=4, header=0, parse_dates={'datetime': ['Date','Time']}, date_parser= lambda x: date_parser_wtd(x), index_col='datetime')


In [58]:
wtd.head()

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,block6,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
datetime,,,,,,,,,,,,,,,,,,,,,
2020-05-12 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-89.930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-89.930,NaN,NaN
2020-05-12 15:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-89.930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-89.930,NaN,NaN
2020-05-12 16:00:00,NaN,NaN,83.807,NaN,NaN,NaN,NaN,NaN,NaN,-90.092,...,-83.807,NaN,NaN,NaN,NaN,NaN,NaN,-90.092,NaN,NaN
2020-05-12 16:30:00,NaN,NaN,83.807,NaN,NaN,NaN,NaN,NaN,NaN,-90.092,...,-83.807,NaN,NaN,NaN,NaN,NaN,NaN,-90.092,NaN,NaN
2020-05-12 17:00:00,NaN,NaN,83.807,NaN,NaN,NaN,NaN,NaN,NaN,-90.092,...,-83.807,NaN,NaN,NaN,NaN,NaN,NaN,-90.092,NaN,NaN


In [59]:
ind_wtd_in_all = wtd.index.get_indexer(time_all, method='nearest')

In [60]:
for key in biomet.keys():
    temp_var = np.zeros((df_all.shape[0],))*np.nan
    temp_var[ind_biomet_in_all] = biomet[key].values
    xr_all[key] = ("time", temp_var)

In [61]:
wtd.columns

Index(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13', 'block3', 'cc_ac', 'block6',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25'],
      dtype='object')

In [62]:
wtd_data = wtd.iloc[ind_wtd_in_all, :].loc[:, ['block6', 'block3', 'cc_ac']]

In [63]:
wtd_data = wtd_data.rename(columns={'cc_ac': 'wtd_automatic_chambers','block6': 'wtd_odyssey_block6', 'block3': 'wtd_odyssey_block3'})

### Interpolate WTD data. Note that this means that starting 29.7.2022 the data is the same number -32.1

In [64]:
for col in wtd_data.columns: 

    xr_all[col] = (["time"], wtd_data.loc[:, col].to_numpy(),{'description':  f'WTD from odyssey logger (cm below soil surface)', 'unit': 'cm'})

## Continuous soil T and moisture

In [65]:
wtd_helena = pd.read_excel(f'{ONEDRIVE_FOLDER}/continuous_soil_T_moisture/cc_site_environmental_data_Helena.xlsx', header=0, index_col='datetime')

In [66]:
wtd_helena

,T5_cc,sm5_cc,wtl_block3_cc,wtl_block6_cc,precipitation_mm
datetime,,,,,
2019-08-28 00:00:00,NaN,NaN,NaN,NaN,NaN
2019-08-28 00:30:00,NaN,NaN,NaN,NaN,NaN
2019-08-28 01:00:00,NaN,NaN,NaN,NaN,NaN
2019-08-28 01:30:00,NaN,NaN,NaN,NaN,NaN
2019-08-28 02:00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2023-07-31 21:30:00,15.74583,0.275949,NaN,NaN,0.0
2023-07-31 22:00:00,15.45606,0.275949,NaN,NaN,0.0
2023-07-31 22:30:00,15.13506,0.275949,NaN,NaN,0.0


In [67]:
wtd_helena_ind_in_all = wtd_helena.index.get_indexer(time_all, method='nearest')

In [68]:
soil_moisture = wtd_helena.iloc[wtd_helena_ind_in_all, :]

In [69]:
soil_moisture.head()

,T5_cc,sm5_cc,wtl_block3_cc,wtl_block6_cc,precipitation_mm
datetime,,,,,
2021-11-24 00:00:00,-0.055222,0.32,-7.000000,-27.325000,0.08
2021-11-24 00:30:00,-0.043907,0.32,-7.008333,-27.347917,0.00
2021-11-24 01:00:00,-0.036000,0.32,-7.016667,-27.370833,0.07
2021-11-24 01:30:00,-0.030167,0.32,-7.025000,-27.393750,0.00
2021-11-24 02:00:00,-0.025611,0.32,-7.033333,-27.416667,0.00


In [70]:
descriptions = [{'description':  'Soil temperature at 5 cm below surface from automated chambers', 'unit': 'C'},
                {'description':  'Soil moisture at 5 cm below surface from automated chambers', 'unit': 'm3m-3'},
                {'description':  'WTD from block 3 odyssey loggers from Helena Rautakoski', 'unit': 'cm'},
                {'description':  'WTD from block 6 odyssey loggers from Helena Rautakoski', 'unit': 'cm'},
                {'description':  'Precipitation from tower at clearcut site', 'unit': 'mm'}
                ]

var_names = ['T_soil_ac','soil_moisture_ac','wtd_helena_block_3', 'wtd_helena_block_6','precipitation']
for i,col in enumerate(soil_moisture.columns):
    xr_all[var_names[i]] = (["time"], soil_moisture.loc[:, col], descriptions[i])

## TOMST soil T and moisture

In [71]:
def tomst_date_parser(x):
    # Date in tomst format is
    # 2021-10-01T00:00:00Z
    date = datetime.strptime(x, r'%Y-%m-%dT%H:%M:%SZ')

    return date

In [72]:
wtd = pd.read_excel(f'{ONEDRIVE_FOLDER}/WTD/ranskalankorpi_odyssey_data/Odyssey_wt-data_ranskalankorpi_2020-05-12_2023-04-27.xlsx', skiprows=4, header=0, parse_dates={'datetime': ['Date','Time']}, date_parser= lambda x: date_parser_wtd(x), index_col='datetime')

/var/folders/_7/7fgf24sd5ggbyr5x9kzrwz7c0000gn/T/ipykernel_18766/1193253790.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  wtd = pd.read_excel(f'{ONEDRIVE_FOLDER}/WTD/ranskalankorpi_odyssey_data/Odyssey_wt-data_ranskalankorpi_2020-05-12_2023-04-27.xlsx', skiprows=4, header=0, parse_dates={'datetime': ['Date','Time']}, date_parser= lambda x: date_parser_wtd(x), index_col='datetime')


In [73]:
tomst = pd.read_csv(f'{ONEDRIVE_FOLDER}/continuous_soil_T_moisture/aggregateData_30min.csv', parse_dates = {'datetime': ['time_30min']}, date_parser=lambda x: tomst_date_parser(x), index_col='datetime')

/var/folders/_7/7fgf24sd5ggbyr5x9kzrwz7c0000gn/T/ipykernel_18766/4169120506.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  tomst = pd.read_csv(f'{ONEDRIVE_FOLDER}/continuous_soil_T_moisture/aggregateData_30min.csv', parse_dates = {'datetime': ['time_30min']}, date_parser=lambda x: tomst_date_parser(x), index_col='datetime')


In [74]:
tomst = tomst.query('Canopy_treatment == "clearcut" and Disturbance_treatment == "Control"')

In [75]:
tomst['Plot'] = tomst['Plot'].astype(np.int32)

In [76]:
tomst.head()

,date,Plot,Canopy_treatment,Disturbance_treatment,T1_soil,T2_surface,T3_air,moist
datetime,,,,,,,,
2021-10-01 00:00:00,2021-10-01,2,clearcut,Control,7.94,6.12,5.88,55.40
2021-10-01 00:30:00,2021-10-01,2,clearcut,Control,7.91,5.84,5.34,55.40
2021-10-01 01:00:00,2021-10-01,2,clearcut,Control,7.88,4.81,3.53,55.40
2021-10-01 01:30:00,2021-10-01,2,clearcut,Control,7.88,4.00,2.62,55.40
2021-10-01 02:00:00,2021-10-01,2,clearcut,Control,7.84,3.81,2.91,55.45


In [77]:
tomst.Plot.unique()

array([2, 3, 4], dtype=int32)

### Loop through the plots and add to dataframe

In [78]:
for plot in tomst.Plot.unique():
    tomst_temp = tomst.query(f'Plot == {plot}')
    tomst_temp = tomst_temp.assign(row_number = np.arange(tomst_temp.shape[0]))
    tomst_temp = tomst_temp.loc[tomst_temp.row_number % 49 != 0]
    ind = tomst_temp.index.get_indexer(time_all, method='nearest')
    tomst_temp = tomst_temp.iloc[ind, :].loc[:, ['T1_soil','T2_surface','T3_air','moist']]
    tomst_temp = tomst_temp.rename(columns={'T1_soil':'T_soil_tomst', 'T2_surface':'T_surface_tomst','T3_air': 'T_air_tomst', 'moist': 'soil_moisture_tomst'})

    start_desc = ['Soil temperature at 6 cm below surface Tomst','Surface temperature Tomst','Air temperature Tomst','Soil moisture Tomst']
    units = ['C','C','C','cm m-1']

    for i,column in enumerate(tomst_temp.columns):
        # name the new dataarray
        data_name = f'{column}_plot_{plot}'
        desc_temp = start_desc[i] + f' plot {plot}' 
        xr_all[data_name] = (['time'], tomst_temp.loc[:, column].to_numpy(), {'Description': desc_temp, 'unit': units[i]})


## Save the created array

In [79]:
#xr_all.to_netcdf(f'{ONEDRIVE_FOLDER}/ec_biomet_wtd_2021_2022_footprints_nan_all_times.nc', format='NETCDF4')
xr_all.to_netcdf(f'{ONEDRIVE_FOLDER}/ec_biomet_wtd_2022_20240607.nc', format='NETCDF4')